In [1]:
%load_ext lab_black

In [2]:
import pandas as pd
from datawrapper import Datawrapper

dw = Datawrapper(
    access_token="FtIwtvFtoGLaRT9a3gjX69PLu4wSuRyKddoOz6SOPw3k9wWyNICMHTkcPhOGCR5Z"
)

In [3]:
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 1000
pd.set_option("display.max_columns", None)

### get data from who

In [4]:
from datetime import date

today = date.today()
enddate = today.strftime("%Y-%m-%d")

In [5]:
url = (
    "https://extranet.who.int/ssa/Ref/GetDashboardFullReport?Countries=229&Types=0&DaterangeSTART=2022-02-24&DaterangeEND="
    + enddate
)

In [6]:
hc_attacks = pd.read_json(url)

### clean up columns

In [7]:
hc_attacks.columns = hc_attacks.columns.str.replace(" ", "_").str.lower()

In [8]:
hc_attacks.attackdate = pd.to_datetime(hc_attacks.attackdate, dayfirst=True)

In [9]:
confirmed_hc_attacks_bydate = (
    hc_attacks[hc_attacks["classification"] == "Confirmed"]
    .groupby("attackdate")
    .size()
    .reset_index(name="count")
)

In [10]:
# can use this to check weekly cumulative matches the daily cumulative BUT don't use to join
# will be missing data for week dates that didn't have attacks--later try to fill in gaps in cumulative sum dataframes
cumulative_hc_attacks = (
    confirmed_hc_attacks_bydate.groupby("attackdate")
    .sum()["count"]
    .cumsum()
    .reset_index(name="attacks")
)

In [11]:
weekly_confirmed_hc_attacks = (
    confirmed_hc_attacks_bydate.groupby(pd.Grouper(key="attackdate", freq="W-SUN"))[
        "count"
    ]
    .sum()
    .reset_index(name="attacks")
)

In [12]:
weekly_cumulative = (
    weekly_confirmed_hc_attacks.groupby("attackdate")
    .sum()["attacks"]
    .cumsum()
    .reset_index(name="cumulative")
)

In [13]:
weekly_and_cumulative = weekly_confirmed_hc_attacks.merge(
    weekly_cumulative, on="attackdate"
)

In [14]:
weekly_hc_attacks_metrics = (
    hc_attacks[hc_attacks.classification == "Confirmed"]
    .groupby(pd.Grouper(key="attackdate", freq="W-SUN"))
    .sum()
    .reset_index()
    .drop(["id", "versionid", "classificationid"], axis=1)
)

In [15]:
weekly_deaths_injuries_dw = weekly_hc_attacks_metrics[
    ["attackdate", "victimstotalinjured", "victimstotaldeath"]
]

### update both in dw

In [16]:
dw.add_data(chart_id="QSTwE", data=weekly_and_cumulative)

<Response [204]>

In [17]:
dw.add_data(chart_id="xyBKc", data=weekly_deaths_injuries_dw)

<Response [204]>

### write data to repository

In [18]:
hc_attacks.to_csv("data/raw/raw_hcattacks.csv")

weekly_and_cumulative.to_csv("data/processed/weekly_hcattacks.csv")

weekly_deaths_injuries_dw.to_csv("data/processed/weekly_hc_injuriesdeaths.csv")